In [1]:
import requests
from bs4 import BeautifulSoup

income_statement = {}

url = "https://finance.yahoo.com/quote/AAPL/financials?p=AAPL"

headers = {"User-Agent" : "Chrome/96.0.4664.110"}
page = requests.get(url, headers=headers)
page_content = page.content
soup = BeautifulSoup(page_content,"html.parser")
tabl = soup.find_all("div" , {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
for t in tabl:
    rows = t.find_all("div" , {"class": "D(tbr) fi-row Bgc($hoverBgColor):h"})
    for row in rows:
        income_statement[row.get_text(separator="|").split("|")[0]] = row.get_text(separator="|").split("|")[1]
        


In [2]:
income_statement

{'Total Revenue': '394,328,000',
 'Cost of Revenue': '223,546,000',
 'Gross Profit': '170,782,000',
 'Operating Expense': '51,345,000',
 'Operating Income': '119,437,000',
 'Net Non Operating Interest Income Expense': '-106,000',
 'Other Income Expense': '-228,000',
 'Pretax Income': '119,103,000',
 'Tax Provision': '19,300,000',
 'Net Income Common Stockholders': '99,803,000',
 'Diluted NI Available to Com Stockholders': '99,803,000',
 'Basic EPS': '-',
 'Diluted EPS': '-',
 'Basic Average Shares': '-',
 'Diluted Average Shares': '-',
 'Total Operating Income as Reported': '119,437,000',
 'Total Expenses': '274,891,000',
 'Net Income from Continuing & Discontinued Operation': '99,803,000',
 'Normalized Income': '99,803,000',
 'Interest Income': '2,825,000',
 'Interest Expense': '2,931,000',
 'Net Interest Income': '-106,000',
 'EBIT': '122,034,000',
 'EBITDA': '133,138,000',
 'Reconciled Cost of Revenue': '223,546,000',
 'Reconciled Depreciation': '11,104,000',
 'Net Income from Conti

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

tickers = ["AAPL", "FB", "CSCO", "INFY.NS", "3988.HK"]
income_statatement_dict = {}
balance_sheet_dict = {}
cashflow_st_dict = {}

for ticker in tickers:
    # scraping income statement
    url = "https://finance.yahoo.com/quote/{}/financials?p={}".format(
        ticker, ticker)
    income_statement = {}
    table_title = {}

    headers = {"User-Agent": "Chrome/96.0.4664.110"}
    page = requests.get(url, headers=headers)
    page_content = page.content
    soup = BeautifulSoup(page_content, "html.parser")
    tabl = soup.find_all(
        "div", {"class": "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
    for t in tabl:
        heading = t.find_all("div", {"class": "D(tbr) C($primaryColor)"})
        for top_row in heading:
            table_title[top_row.get_text(separator="|").split(
                "|")[0]] = top_row.get_text(separator="|").split("|")[1:]
        rows = t.find_all(
            "div", {"class": "D(tbr) fi-row Bgc($hoverBgColor):h"})
        for row in rows:
            income_statement[row.get_text(separator="|").split(
                "|")[0]] = row.get_text(separator="|").split("|")[1:]

    temp = pd.DataFrame(income_statement).T
    temp.columns = table_title["Breakdown"]
    income_statatement_dict[ticker] = temp

    # scraping balance sheet statement
    url = "https://finance.yahoo.com/quote/{}/balance-sheet?p={}".format(
        ticker, ticker)
    balance_sheet = {}
    table_title = {}

    headers = {"User-Agent": "Chrome/96.0.4664.110"}
    page = requests.get(url, headers=headers)
    page_content = page.content
    soup = BeautifulSoup(page_content, "html.parser")
    tabl = soup.find_all(
        "div", {"class": "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
    for t in tabl:
        heading = t.find_all("div", {"class": "D(tbr) C($primaryColor)"})
        for top_row in heading:
            table_title[top_row.get_text(separator="|").split(
                "|")[0]] = top_row.get_text(separator="|").split("|")[1:]
        rows = t.find_all(
            "div", {"class": "D(tbr) fi-row Bgc($hoverBgColor):h"})
        for row in rows:
            balance_sheet[row.get_text(separator="|").split(
                "|")[0]] = row.get_text(separator="|").split("|")[1:]

    temp = pd.DataFrame(balance_sheet).T
    temp.columns = table_title["Breakdown"]
    balance_sheet_dict[ticker] = temp

    # scraping cashflow statement
    url = "https://finance.yahoo.com/quote/{}/cash-flow?p={}".format(
        ticker, ticker)
    cashflow_statement = {}
    table_title = {}

    headers = {"User-Agent": "Chrome/96.0.4664.110"}
    page = requests.get(url, headers=headers)
    page_content = page.content
    soup = BeautifulSoup(page_content, "html.parser")
    tabl = soup.find_all(
        "div", {"class": "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
    for t in tabl:
        heading = t.find_all("div", {"class": "D(tbr) C($primaryColor)"})
        for top_row in heading:
            table_title[top_row.get_text(separator="|").split(
                "|")[0]] = top_row.get_text(separator="|").split("|")[1:]
        rows = t.find_all(
            "div", {"class": "D(tbr) fi-row Bgc($hoverBgColor):h"})
        for row in rows:
            cashflow_statement[row.get_text(separator="|").split(
                "|")[0]] = row.get_text(separator="|").split("|")[1:]

    temp = pd.DataFrame(cashflow_statement).T
    temp.columns = table_title["Breakdown"]
    cashflow_st_dict[ticker] = temp

# converting dataframe values to numeric
for ticker in tickers:
    for col in income_statatement_dict[ticker].columns:
        income_statatement_dict[ticker][col] = income_statatement_dict[ticker][col].str.replace(
            ',|- ', '')
        income_statatement_dict[ticker][col] = pd.to_numeric(
            income_statatement_dict[ticker][col], errors='coerce')
        cashflow_st_dict[ticker][col] = cashflow_st_dict[ticker][col].str.replace(
            ',|- ', '')
        cashflow_st_dict[ticker][col] = pd.to_numeric(
            cashflow_st_dict[ticker][col], errors='coerce')
        if col != "ttm":  # yahoo has ttm column for income statement and cashflow statement only
            balance_sheet_dict[ticker][col] = balance_sheet_dict[ticker][col].str.replace(
                ',|- ', '')
            balance_sheet_dict[ticker][col] = pd.to_numeric(
                balance_sheet_dict[ticker][col], errors='coerce')


/var/folders/q4/72ybtynd5sjf5ppl_5vpsw080000gn/T/ipykernel_57904/2834662776.py:80: FutureWarning: The default value of regex will change from True to False in a future version.
  income_statatement_dict[ticker][col] = income_statatement_dict[ticker][col].str.replace(',|- ','')
/var/folders/q4/72ybtynd5sjf5ppl_5vpsw080000gn/T/ipykernel_57904/2834662776.py:82: FutureWarning: The default value of regex will change from True to False in a future version.
  cashflow_st_dict[ticker][col] = cashflow_st_dict[ticker][col].str.replace(',|- ','')
/var/folders/q4/72ybtynd5sjf5ppl_5vpsw080000gn/T/ipykernel_57904/2834662776.py:85: FutureWarning: The default value of regex will change from True to False in a future version.
  balance_sheet_dict[ticker][col] = balance_sheet_dict[ticker][col].str.replace(',|- ','')


In [12]:
balance_sheet.keys()

dict_keys(['Total Assets', 'Total Liabilities Net Minority Interest', 'Total Equity Gross Minority Interest', 'Total Capitalization', 'Preferred Stock Equity', 'Common Stock Equity', 'Net Tangible Assets', 'Invested Capital', 'Tangible Book Value', 'Total Debt', 'Share Issued', 'Ordinary Shares Number'])

Getting key statistics data from yahoo finance using webscraping

In [13]:
import requests
from bs4 import BeautifulSoup

tickers = ["AAPL","FB","CSCO","INFY.NS","3988.HK"]
key_statistics = {}

for ticker in tickers:
    #scraping key statistics
    url = "https://finance.yahoo.com/quote/{}/key-statistics?p={}".format(ticker,ticker)    
    headers = {"User-Agent" : "Chrome/96.0.4664.110"}
    page = requests.get(url, headers=headers)
    page_content = page.content
    soup = BeautifulSoup(page_content,"html.parser")
    tabl = soup.find_all("table" , {"class" : "W(100%) Bdcl(c)"}) #remove/add the trailing space if getting error
    
    temp_stats = {}
    for t in tabl:
        rows = t.find_all("tr")
        for row in rows:
            temp_stats[row.get_text(separator="|").split("|")[0]] = row.get_text(separator="|").split("|")[-1]
    
    key_statistics[ticker] = temp_stats

In [15]:
key_statistics["AAPL"]

{'Beta (5Y Monthly)': '1.22',
 '52-Week Change': '-26.88%',
 'S&P500 52-Week Change': '-19.75%',
 '52 Week High': '182.94',
 '52 Week Low': '129.04',
 '50-Day Moving Average': '144.13',
 '200-Day Moving Average': '152.13',
 'Avg Vol (3 month)': '86.54M',
 'Avg Vol (10 day)': '89.02M',
 'Shares Outstanding': '15.91B',
 'Implied Shares Outstanding': 'N/A',
 'Float': '15.89B',
 '% Held by Insiders': '0.07%',
 '% Held by Institutions': '60.03%',
 'Shares Short (Nov 29, 2022)': '125.72M',
 'Short Ratio (Nov 29, 2022)': '1.52',
 'Short % of Float (Nov 29, 2022)': '0.79%',
 'Short % of Shares Outstanding (Nov 29, 2022)': '0.79%',
 'Shares Short (prior month Oct 30, 2022)': '103.18M',
 'Forward Annual Dividend Rate': '0.92',
 'Forward Annual Dividend Yield': '0.70%',
 'Trailing Annual Dividend Rate': '0.90',
 'Trailing Annual Dividend Yield': '0.68%',
 '5 Year Average Dividend Yield': '0.99',
 'Payout Ratio': '14.73%',
 'Dividend Date': 'Nov 09, 2022',
 'Ex-Dividend Date': 'Nov 03, 2022',
 'La